In [49]:
import time

In [50]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [51]:
wd = "D:\\Users\\kimdu\\Desktop\\GRAD\\chromedriver.exe"
addr = "https://news.nate.com/view/20200225n34937?mid=n1008"

In [52]:
driver = webdriver.Chrome(wd)
driver.get(addr)

In [53]:
pages = 0

In [54]:
try:
    while True:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"body > div > div > div.paging_wrap > a"))).click()
        time.sleep(1.5)
        print(pages, end=" ")
        pages+=1
except exceptions.ElementNotVisibleException as e: # 페이지 끝
    pass
    
except Exception as e: # 다른 예외 발생시 확인
    print(e)

Message: 



In [55]:
html = driver.page_source
dom = BeautifulSoup(html, "lxml")

In [56]:
# 댓글이 들어있는 페이지 전체 크롤링
comments_raw = dom.find_all("span", {"class" : "usertxt"})
# 댓글의 text만 뽑는다.
comments = [comment.text for comment in comments_raw]


In [57]:
comments[:3]

[]

In [58]:
p_comment_list = []

In [59]:
for comment in comments:
    p_comment_list.append(comment)

In [60]:
file = open('p_comment44.txt','w',encoding = 'utf-8')
for comment in p_comment_list:
    file.write(comment + '\n')
file.close()